In [11]:
import torch

In [12]:
model = torch.load("../Models/lif_snn_mnist.ckpt")
model.eval()

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=256, out_features=64, bias=False)
  (2): LIFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, tau=2.0
    (surrogate_function): Sigmoid(alpha=1.0, spiking=True)
  )
  (3): Linear(in_features=64, out_features=10, bias=False)
  (4): LIFNode(
    v_threshold=1.0, v_reset=0.0, detach_reset=False, tau=2.0
    (surrogate_function): Sigmoid(alpha=1.0, spiking=True)
  )
)

In [13]:
w_12 = model[1].weight.detach().cpu().numpy()
w_23 = model[3].weight.detach().cpu().numpy()

In [14]:
print(w_12.shape, w_23.shape)

(64, 256) (10, 64)


In [15]:
f = open('../Models/weights1.txt', 'w')
for i in range(256):
    for j in range(64):
        print(w_12[j][i], file = f, end = ' ')
    print("", file = f)
f.close()

In [16]:
f = open('../Models/weights2.txt', 'w')
for i in range(64):
    for j in range(10):
        print(w_23[j][i], file = f, end = ' ')
    print("", file = f)
f.close()

In [7]:
import torchvision

test_dataset = torchvision.datasets.MNIST(
        root="../Datasets/",
        train=False,
        download=True,
        transform=torchvision.transforms.Compose([
            torchvision.transforms.Resize(16),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(
                (0.1307,), (0.3081,))
        ])
    )

test_data_loader = torch.utils.data.DataLoader(
        dataset=test_dataset,
        batch_size=64,
        shuffle=False
)

In [8]:
test_data_loader

In [22]:
from spikingjelly.clock_driven import encoding
encoder = encoding.PoissonEncoder()
cnt = 0
f1 = open('../Datasets/MNIST/input.txt', 'w')
for img, label in test_data_loader:
    cnt += 1
    #if (cnt > 1):
    #    break;
    rg = 64
    if (cnt > 156):
        rg = 16
    for i in range(rg):
        for j in range(16):
            for k in range(16): 
                print(int(encoder(img[i][0][j][k]).float().tolist()), file = f1, end = ' ')
        print("", file = f1)
f1.close()
print(cnt * 64)

10048


In [26]:
f2 = open('../Datasets/MNIST/output.txt', 'w')
cnt = 0;
for img, label in test_data_loader:
    cnt += 1
    #if (cnt > 1):
    #    break;
    rg = 64
    if (cnt > 156):
        rg = 16
    for i in range(rg):
        a = int(label[i].tolist())
        print(a, file = f2)
f2.close()

In [22]:
import math
device = torch.device('cpu')

for p in model[1].parameters():
    for v in p.data:
        print(v)

x = torch.Tensor([-1, -2, -3], device = device)
x.apply_(lambda x: x if x >= 0 else 0.01 * math.exp(x))
for v in x:
    v = -v
    print(v)

tensor([ 4.1591e-02,  2.0918e-02,  3.4410e-02,  3.4491e-02,  2.5918e-01,
         3.0539e-01,  6.5578e-01,  7.9293e-01,  3.2252e-01,  6.9019e-01,
         3.8822e-01,  2.7676e-01,  1.1174e-01,  6.6989e-02,  2.6438e-02,
         4.2336e-02,  4.4998e-03,  1.7811e-02,  5.0021e-02,  5.0633e-01,
         1.0503e+00,  1.9709e+00,  3.1496e+00,  3.7741e+00,  3.4632e+00,
         3.1082e+00,  3.5231e+00,  2.9282e+00,  1.7723e+00,  9.0489e-01,
         2.2064e-01,  7.2677e-03,  5.9439e-02,  1.0572e-02,  9.6620e-04,
         3.8709e-01,  1.1453e+00,  1.7053e+00,  6.8491e-01,  9.9319e-02,
         5.4862e-02,  3.6911e-02, -1.3917e-01, -1.0108e-01, -8.9194e-02,
        -8.7479e-02,  4.9489e-02, -3.6073e-02,  6.2070e-02,  6.1432e-02,
         1.5106e-01,  1.3468e-01,  3.1713e-01,  6.5660e-01,  2.0554e-01,
        -4.7897e-02, -1.1626e-01,  2.1623e-02, -1.4975e-01, -2.0000e-01,
        -2.0105e-01, -3.0646e-01, -3.5328e-01,  5.9605e-02,  5.7286e-03,
         9.6233e-02,  9.1122e-02,  3.1573e-01,  4.4

tensor([ 3.2025e-02,  5.3462e-02,  4.7431e-02,  2.5630e-03, -1.2346e-02,
        -7.3809e-02, -1.4300e-01, -2.7098e-02,  6.5994e-03,  1.0963e+00,
        -5.3368e-02,  2.1916e-02,  5.4831e-02,  3.1247e-01,  3.5600e-02,
         5.1023e-02,  5.6122e-02,  6.7101e-03,  1.7615e-03,  2.3692e-02,
         4.3574e-03, -6.4670e-02,  1.5966e-02, -2.3429e-02, -2.1690e-02,
         4.5973e-02,  6.6493e-02,  3.7331e-02, -2.8491e-02,  1.8809e-02,
         6.1394e-01,  1.8815e-02,  1.2572e-02,  3.7946e-03, -2.5118e-02,
         1.9558e+00,  4.9048e-01,  8.8930e-01,  4.7483e-01,  5.6427e-01,
         3.7645e-01,  7.4337e-01,  1.4341e-01, -8.7247e-02, -2.0165e-01,
        -2.3724e-01, -4.6011e-02, -6.3011e-04,  4.3848e-02, -6.4432e-02,
         5.3187e-01,  9.7520e-01,  3.2310e-01,  5.6353e-02,  7.5303e-03,
        -2.5276e-02, -1.3265e-02, -9.1280e-03, -7.3393e-03, -3.8195e-02,
        -4.0617e-02, -1.2480e-01, -1.0408e-01,  1.3212e-02, -1.5128e-02,
         4.7985e-01,  1.3767e-01,  9.4466e-01,  2.1

tensor([ 3.9256e-02,  1.2221e-02,  5.6938e-02,  1.1253e-01,  1.2095e-02,
         1.1216e-04,  8.6608e-03,  3.4384e-02,  4.6516e-02,  5.2752e-02,
         3.8642e-03,  4.0065e-02,  1.6925e-02,  2.1372e-03,  2.0005e-02,
         2.9261e-02,  4.2540e-02,  2.6271e-02,  1.0671e-01,  2.8763e-01,
         1.1154e-01,  2.5403e-01,  1.7012e-01,  2.1658e-01,  5.4581e-03,
         1.0097e-01,  1.6366e-01,  1.9396e-01,  1.8007e-01,  1.2398e+00,
         5.3757e-01,  5.9145e-02,  4.7533e-03,  9.6495e-03,  2.5686e-02,
         1.5216e-01,  1.3045e-01,  2.4148e-01,  2.3906e-01,  1.7254e-01,
         2.7728e-01,  2.6953e-01,  3.0890e-01,  6.2746e-01,  4.9674e-01,
         1.6405e-01,  1.0874e-01,  3.3483e-02,  2.0408e-02,  1.5547e-02,
         4.8189e-01,  9.2572e-01,  1.0011e+00,  8.2827e-01,  2.1088e-01,
         3.7123e-01,  4.6358e-01,  4.2643e-01,  3.1314e-01,  3.6250e-01,
         4.4617e-01,  1.0903e+00,  1.0730e-01,  5.1086e-02,  7.6907e-02,
         2.5082e-01,  2.4051e-01,  4.2312e-01,  4.8

tensor([ 2.8144e-02,  5.3767e-02,  3.0633e-02,  3.0900e-01,  4.8320e-01,
         2.8329e-02,  2.5774e-01,  6.6047e-01,  3.9527e-01,  7.6457e-01,
         1.4291e+00,  6.6870e-01,  3.7266e-01,  3.2488e-01,  3.2151e-02,
         3.3909e-02,  2.5782e-02,  5.7936e-02,  2.7499e-01,  7.6157e-01,
         7.0697e-01,  2.4067e-01,  8.2770e-01,  5.5945e-02,  1.5183e-01,
         1.4043e-01,  3.2106e-01,  4.5594e-01,  1.2286e+00,  2.4073e+00,
         9.9971e-01,  2.9145e-02,  4.2256e-02,  1.2954e-02,  7.4851e-02,
        -3.6397e-02, -9.1859e-02, -1.8706e-02, -2.9769e-02, -5.7082e-02,
        -4.3602e-02, -1.2203e-01, -1.1808e-01, -8.8326e-02, -4.8923e-02,
         3.7786e-02,  5.4786e-03,  1.6237e-02,  1.1188e-02,  1.3433e-01,
        -2.6877e-02, -5.3521e-02, -6.1109e-02, -5.9638e-02, -6.0369e-02,
        -1.0069e-01, -1.2339e-01, -1.3060e-01, -1.4602e-02,  2.0182e-03,
        -7.0170e-02,  9.1430e-03,  1.4539e-02,  3.9068e-02,  1.4423e-02,
         7.1327e-03, -3.6568e-02, -2.5711e-02,  1.6

tensor([ 1.5701e-02,  4.4690e-02,  2.5295e-02,  1.3176e-02,  4.2827e-02,
         7.4815e-02,  5.8792e-02,  5.0701e-02,  1.4132e-01,  1.4606e-01,
         1.0693e-01,  1.0940e-01,  4.5763e-02,  2.6994e-03,  3.5303e-02,
         1.8807e-02,  2.8888e-02,  7.7511e-02,  1.0627e-01,  4.4843e-02,
         4.8650e-02,  1.0871e-02,  1.3488e-01,  1.9813e-01,  7.8509e-02,
         1.9190e-01,  3.7518e-01,  4.2154e-01,  2.3392e-01,  1.4004e-01,
         2.2248e-01,  2.3206e-02,  3.6836e-03,  9.0434e-02, -1.0414e-02,
         7.9021e-03, -6.9814e-02, -1.2024e-01, -7.3226e-02,  8.1853e-02,
         1.6425e-01,  1.2326e-01,  4.4187e-01,  4.3204e-01,  4.4945e-01,
         2.4801e-01,  7.9300e-02, -3.4382e-03,  5.7633e-02,  1.3170e+00,
         1.8407e-01, -3.4135e-02, -4.0737e-02, -7.3167e-03, -2.3066e-01,
        -1.6306e-01, -2.1334e-01, -3.3155e-01, -8.6839e-02,  4.9354e-01,
         7.6806e-01,  5.9352e-01,  3.0627e-01,  1.2334e-01,  7.5065e-01,
         2.9227e+00,  2.8193e-01,  1.7104e-02, -1.3

tensor([ 2.6772e-02,  2.8914e-02,  5.6332e-02,  2.5160e-01,  2.1689e-01,
         2.3971e-01,  3.3127e-01,  3.4489e-01,  2.4528e-02,  3.3277e-01,
         9.6019e-01,  4.7448e-01,  2.3002e-01,  2.6302e-01,  5.0795e-02,
         2.2155e-02,  7.7387e-03,  5.1904e-02,  3.5008e-01,  9.2524e-01,
         1.1492e+00,  4.8968e-01,  4.5083e-01,  2.2514e-01,  3.5387e-02,
        -1.3439e-02,  2.4535e-01,  3.8506e-01,  1.0819e+00,  1.8493e+00,
         9.2661e-01,  3.4681e-02,  5.7673e-02, -1.6717e-03,  6.3009e-02,
        -4.6824e-02, -9.7763e-02, -3.4187e-02, -8.0466e-02, -1.0266e-01,
        -1.0467e-01, -3.3146e-02, -4.0235e-02,  5.2889e-02,  1.4095e-01,
         2.9867e-01,  1.7792e-01,  2.2200e-01,  5.8966e-02,  2.1925e-01,
        -1.0497e-02, -6.9773e-02, -5.1122e-02,  1.2348e-02, -7.8069e-02,
        -1.0789e-01, -1.2598e-01, -8.6547e-02, -9.7520e-02,  6.1437e-02,
         1.3109e-01,  3.6025e-01,  2.0306e-01,  7.7013e-02,  9.9911e-02,
         1.6876e-01,  7.8444e-02, -4.3820e-02, -1.0

tensor([ 1.8852e-02,  5.3835e-02,  1.9550e-02,  1.0808e-02,  8.6574e-02,
         1.1296e-01,  2.5872e-01,  3.4116e-01,  8.9143e-02,  2.2174e-01,
         1.8136e-01,  1.9573e-01,  5.2381e-02,  5.0307e-02,  4.5829e-02,
         9.2178e-03,  7.4153e-03,  8.6785e-04,  3.1745e-02,  5.1951e-03,
         1.0789e-01,  4.6826e-01,  9.8078e-01,  1.2200e+00,  8.3330e-01,
         3.9095e-01,  8.9197e-01,  1.2360e+00,  8.2298e-01,  4.2791e-01,
         8.1315e-02,  1.7197e-02,  1.5268e-02,  1.8990e-02,  5.5814e-02,
         1.0563e-02,  4.0392e-02,  3.5832e-01,  7.1262e-01,  6.9174e-01,
         4.3393e-01,  3.3756e-01,  1.3027e-01,  5.5120e-02,  2.0800e-02,
         3.3060e-01,  2.7613e-01,  6.7455e-02,  2.6253e-02,  2.7169e-02,
         5.0761e-01,  3.6864e-01,  3.8434e-02,  6.3721e-02,  2.3177e-01,
         7.9512e-01,  1.2232e+00,  8.0303e-01,  1.1688e-01,  3.1610e-01,
         4.7866e-01,  1.7184e-01,  3.3402e-02,  1.2843e-01,  3.2784e-02,
         6.1096e-02,  1.3507e-01,  3.7333e-01,  3.9

tensor([ 2.6199e-02,  1.5076e-02,  2.9288e-02,  1.8344e+00,  4.2281e+00,
         5.3538e+00,  6.9145e+00,  5.7830e+00,  3.4586e+00, -3.2936e-02,
         4.3704e+00,  3.3641e+00,  2.0424e+00,  1.7747e+00,  1.8021e-04,
         7.6681e-04,  2.5055e-02,  3.5892e-01,  1.8015e+00,  3.2450e+00,
         6.5226e+00,  6.3056e+00,  5.6240e+00,  4.0763e+00,  5.5737e+00,
         5.8054e+00,  8.4469e+00,  8.2485e+00,  9.7135e+00,  6.8699e+00,
         1.6224e-02,  6.1009e-03,  2.8241e-02, -1.2834e-02,  5.8047e-01,
         4.4623e-02,  2.5965e-02,  6.6040e-02,  1.2923e-01,  1.1388e-01,
         6.3571e-02,  6.1998e-01,  5.5668e-01,  7.8462e-01,  6.0648e-01,
         1.0113e+00,  9.9030e-01,  1.2944e+00,  4.5368e-02, -1.2480e-02,
         6.2715e-02,  6.5128e-02,  7.1838e-02,  8.6117e-02,  4.6882e-02,
         4.5884e-02,  5.5797e-02,  7.1624e-02,  5.7873e-02, -1.0478e-03,
         7.4177e-03,  1.3136e-02,  7.9516e-02,  1.1958e+00, -1.7398e-02,
         2.9346e-02,  5.2331e-02,  1.1868e-01,  8.2